## Algorithmic Design Homework 3

### Exercise 1.
Implement the binary heap-based version of the Dijkstra’s algorithm.

### Solution

In the cell below we implement the heap-based version of the Dijkstra’s algorithm on the Graph data structure written in the file `Graph.py` and using the binary heap defined in `Heap.py`.  

The heap implements all the methods we saw during the lectures and some more which we found useful during the implementation of the Dijkstra's algorithm.  

However designing the graph was more complicated because it can be implemented in different ways, we chosed to define a Vertex class which contains a dictionary with its outward connections and their weight i.e. `connections={destination: weight}`, furthermore each vertex has a name which is unique in the graph, the definition of the vertex as an object allows us to add attributes to the vertexes very easily, like the color in breadth/depth first search or the distance in the Dijkstra algorithm. Finally we encoded the graph as a dictionary with key the vertex name and value an instance of the Vertex class, we chosed the dictionary because the time it takes to find a vertex given a name is $\Theta(1)$

In [15]:
from numpy import inf
from src import *
from warnings import warn
from random import randint


def init_sssp(G, s):
    u = G.get_vertex(s)
    if u is None:
        message = f"The vertex{s} is not present in the graph, exiting"
        warn(message, RuntimeWarning)
        return
    
    for v in G.V():
        v.d = inf
        v.pred = None
        
    u.d = 0.

    
def relax(Q, u, v):
    w = u.get_weight(v.name)
    if v.d > u.d + w:
        v.d = u.d + w
        v.pred = u
        Q.decrease_value(v, v)

        
def dijkstra(G, s):
    init_sssp(G, s)
    Q = Binary_heap([G.get_vertex(s)], total_order=lambda x, y: x.d < y.d, dict_key=lambda x: x.name)
    while not Q.is_empty():
        u = Q.remove_min()
        for v in G.Adj(u.name):
            v = G.get_vertex(v)
            Q.insert(v)
            relax(Q, u, v)

# Create a simple graph for testing
G = Graph(directed=True)
G.add_edge("A", "B", 1000)
G.add_edge("A", "C", 1)
G.add_edge("C", "D", 5)
G.add_edge("D", "B", 8)
G.add_vertex("E")

dijkstra(G, "A")  

In [12]:
[(v.name, v.d) for v in G.V()]

[('B', 14.0), ('A', 0.0), ('C', 1.0), ('D', 6.0), ('E', inf)]

In [13]:
def init_sssp(G, s):
    u = G.get_vertex(s)
    if u is None:
        message = f"The vertex{s} is not present in the graph, exiting"
        warn(message, RuntimeWarning)
        return
    
    for v in G.V():
        v.d = inf
        v.pred = None
        
    u.d = 0.


def relax(Q, u, v):
    w = u.get_weight(v.name)
    if v.d > u.d + w:
        v.d = u.d + w
        v.pred = u
        Q.decrease_value(v, v)


def dijkstra_iteration(G, Q):
    u = Q.remove_min()
    for v in G.Adj(u.name):
        v = G.get_vertex(v)
        if v.importance < u.importance:
            continue
        Q.insert(v)
        relax(Q, u, v)
    
    return u

        
def Bidirectional_dijkstra(G, s, d):
    G_down = G.T()
    G_up = G
    G_up.add_shortcuts()
    G_down.add_shortcuts()

    init_sssp(G_up, s)
    init_sssp(G_down, d)

    Q_up = Binary_heap([G_up.get_vertex(s)], total_order=lambda x, y: x.d < y.d, dict_key=lambda x: x.name)
    Q_down = Binary_heap([G_down.get_vertex(d)], total_order=lambda x, y: x.d < y.d, dict_key=lambda x: x.name)

    vertices_up = set()
    vertices_down = set()
    while not Q_up.is_empty() and not Q_down.is_empty():
        up = dijkstra_iteration(G_up, Q_up)
        down = dijkstra_iteration(G_down, Q_down)

        vertices_up.add(up)
        vertices_down.add(down)
        if down in vertices_up or up in vertices_down:
            break

    return vertices_down, vertices_up



In [16]:
for v in G.V():
    v.importance=randint(0, 10)

In [18]:
Bidirectional_dijkstra(G, "A", "B")

KeyError: 'B'